In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from scipy import sparse

from tqdm import tqdm
tqdm.pandas()

import pyLDAvis
import pyLDAvis.gensim_models
import gensim

import umap
import umap.plot
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import LatentDirichletAllocation as LDA

import warnings
warnings.simplefilter('ignore')




from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

C:\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
C:\Python310\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [ ]:
def predproccess(s):
    import numpy as np
    
    from nltk import word_tokenize
    from nltk.stem import WordNetLemmatizer 

    from stop_words import get_stop_words
    stop_words = get_stop_words('en')

    s = s.lower()
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(s)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token.isalpha()]
    if len(tokens) >= 3:
        return tokens
    else:
        return np.NAN

In [2]:
# df = pd.read_csv('titles.csv', delimiter='|')
# titles = df.title
# titles.shape
df = pd.read_csv('step_df.csv')

In [ ]:
df

In [ ]:
%%time
df.title = df.title.parallel_apply(predproccess)

In [ ]:
df.dropna(subset=['title'], inplace=True)

In [ ]:
# df._id.to_csv('step_df.csv', index=False)

In [ ]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
text_clean = df.title.to_list()

In [ ]:
df.drop(['title'], axis=1, inplace=True)

In [ ]:
text_clean

In [ ]:
bigram = Phrases(text_clean, connector_words=ENGLISH_CONNECTOR_WORDS)

In [ ]:
for idx in range(len(text_clean)):
    for token in bigram[text_clean[idx]]:
        if '_' in token and token not in text_clean[idx]:
            text_clean[idx].append(token)

In [ ]:
text_clean

In [ ]:
%%time
dictionary = Dictionary(text_clean)
dictionary.filter_extremes(no_below=10, no_above=0.1)
corpus = [dictionary.doc2bow(doc) for doc in text_clean]

In [ ]:
print('Количество уникальных токенов: %d' % len(dictionary))
print('Количество документов: %d' % len(corpus))

In [4]:
# from gensim import corpora
# dictionary.save('dictionary.dict')
# corpora.MmCorpus.serialize('corpus.crp', corpus)

NameError: name 'dictionary' is not defined

In [ ]:
import json
json.dump(text_clean, open('text_clean.json', 'w'))

In [ ]:
del text_clean

In [5]:
from gensim import corpora
dictionary = corpora.Dictionary.load("dictionary.dict")
corpus = corpora.MmCorpus("corpus.crp")

In [ ]:
# import json
# text_clean = json.load(open('text_clean.json', 'r'))

In [ ]:
from gensim.models.ldamulticore import LdaMulticore

In [ ]:
coherence_values = []
models_list = []
x_range = range(2, 22)
for n_topics in tqdm(x_range):
    model=LdaMulticore(corpus=corpus,id2word=dictionary, num_topics=n_topics)
    models_list.append(model)
    coherencemodel = CoherenceModel(model=model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
    coherence_values.append(coherencemodel.get_coherence())

In [ ]:
sns.lineplot(y=coherence_values, x=x_range)

In [ ]:
coherence_values

In [ ]:
n_t = 2
for model in models_list:
    tags = {}
    for i, token_id in enumerate(np.argmax(model.get_topics(), axis=1)):
        tags[i] = dictionary[token_id]
    print(f'{n_t}: {tags}')
    n_t += 1
    

In [3]:
best_model = pickle.load(open('model_10_tags.pkl', 'rb'))

In [24]:
# best_model = models_list[8]
tags = {}
for i, token_id in enumerate(np.argmax(best_model.get_topics(), axis=1)):
    tags[i] = dictionary[token_id]

In [25]:
tags

{0: 'data',
 1: 'study',
 2: 'based',
 3: 'algorithm',
 4: 'learning',
 5: 'graph',
 6: 'network',
 7: 'research',
 8: 'network',
 9: 'deep'}

In [ ]:
%%time
data = pyLDAvis.gensim_models.prepare(best_model, corpus, dictionary)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.display(data)

<img src="2_class.jpg">

In [26]:
tags[2] = 'wireless_traffic'

In [27]:
# pickle.dump(best_model, open('model_10_tags.pkl', 'wb'))

In [28]:
tags

{0: 'data',
 1: 'study',
 2: 'wireless_traffic',
 3: 'algorithm',
 4: 'learning',
 5: 'graph',
 6: 'network',
 7: 'research',
 8: 'network',
 9: 'deep'}

In [ ]:
del corpus
del dictionary

In [ ]:
best_model.num_topics

In [ ]:
best_model.show_topics()

In [ ]:
len(corpus)

In [10]:
def get_tag(corpus_sample):
#     import numpy as np
#     import pickle
#     best_model = pickle.load(open('model_10_tags.pkl', 'rb'))
    
    if corpus_sample:
        res = np.zeros(10)
        pred = best_model.get_document_topics(corpus_sample)
        for i, score in pred:
            res[i] = score
            
        if len(res) == 10 and (res == res[0]).all():
            return np.NAN
        return res.argmax()
    else:
        return np.NAN

In [6]:
%%time
df['corpus_sample'] = list(corpus)

CPU times: total: 41.9 s
Wall time: 50.1 s


In [11]:
df['tag'] = df.corpus_sample.progress_apply(get_tag)

100%|██████████████████████████████████████████████████████████████████████| 5176332/5176332 [14:15<00:00, 6047.61it/s]


In [20]:
df

,_id,tag
0,53e99784b7602d9701f3e133,3.0
1,53e99784b7602d9701f3e151,9.0
2,53e99784b7602d9701f3e15d,4.0
3,53e99784b7602d9701f3e922,7.0
4,53e99784b7602d9701f3f411,2.0
...,...,...
5176327,6087e112e4510cd7c80b9128,1.0
5176328,6087e112e4510cd7c80b912b,7.0
5176329,6087e112e4510cd7c80b912a,1.0
5176330,6087e112e4510cd7c80b912c,7.0


In [ ]:
# %%time
# preds = pd.DataFrame(columns=range(best_model.num_topics))
# for bow in tqdm(corpus):
#     if bow:
#         preds = preds.append(dict(best_model.get_document_topics(bow)), ignore_index=True)
#     else:
#         preds = preds.append([np.NAN], ignore_index=True)

In [29]:
df.dropna(subset=['tag'], inplace=True)
df.tag = df.tag.astype(int)

In [30]:
df

,_id,tag
0,53e99784b7602d9701f3e133,3
1,53e99784b7602d9701f3e151,9
2,53e99784b7602d9701f3e15d,4
3,53e99784b7602d9701f3e922,7
4,53e99784b7602d9701f3f411,2
...,...,...
5176327,6087e112e4510cd7c80b9128,1
5176328,6087e112e4510cd7c80b912b,7
5176329,6087e112e4510cd7c80b912a,1
5176330,6087e112e4510cd7c80b912c,7


In [32]:
df.tag = df.tag.map(tags)

In [33]:
df

,_id,tag
0,53e99784b7602d9701f3e133,algorithm
1,53e99784b7602d9701f3e151,deep
2,53e99784b7602d9701f3e15d,learning
3,53e99784b7602d9701f3e922,research
4,53e99784b7602d9701f3f411,wireless_traffic
...,...,...
5176327,6087e112e4510cd7c80b9128,study
5176328,6087e112e4510cd7c80b912b,research
5176329,6087e112e4510cd7c80b912a,study
5176330,6087e112e4510cd7c80b912c,research


In [34]:
df.tag.value_counts()

network             1495232
wireless_traffic     590890
study                536479
research             457815
deep                 443990
graph                442449
learning             438727
algorithm            429812
data                 339977
Name: tag, dtype: int64

In [35]:
df.to_csv('titles_tag.csv', index=False)

In [37]:
s = pd.read_csv('titles_tag.csv', chunksize=100)

In [40]:
s.get_chunk()

,_id,tag
100,53e99796b7602d9701f5c543,learning
101,53e99796b7602d9701f5c558,study
102,53e99796b7602d9701f5c55d,research
103,53e99796b7602d9701f5c55e,deep
104,53e99796b7602d9701f5c563,graph
...,...,...
195,53e997cbb7602d9701fbd707,network
196,53e997cbb7602d9701fbd176,study
197,53e997cbb7602d9701fbd179,research
198,53e997cbb7602d9701fbd490,wireless_traffic
